In [1]:
# Importando bibliotecas

import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, util

## Análise de Dados

In [2]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv',
                 encoding='latin-1',
                 header=None,
                 names=['target', 'ids', 'date', 'flag', 'user', 'text'])

In [3]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
df.dtypes

target     int64
ids        int64
date      object
flag      object
user      object
text      object
dtype: object

In [5]:
df['date'] = pd.to_datetime(df['date'])

C:\Users\Lucas\AppData\Local\Temp\ipykernel_19776\3532345252.py:1: FutureWarning: Parsed string "Mon Apr 06 22:19:45 PDT 2009" included an un-recognized timezone "PDT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df['date'] = pd.to_datetime(df['date'])


In [6]:
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,2009-04-06 22:19:45,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,2009-04-06 22:19:49,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,2009-04-06 22:19:53,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,2009-04-06 22:19:57,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,2009-04-06 22:19:57,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
df['date'].dt.year.unique()

array([2009])

Todos os tweets são datados de 2009

In [8]:
df['target'].unique()

array([0, 4], dtype=int64)

TARGET: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)

In [9]:
df.shape

(1600000, 6)

DF com 1.6M de linhas e 6 colunas

## Tratamento de Dados

In [10]:
df_treat = df.copy()

In [11]:
df_treat.head()

,target,ids,date,flag,user,text
0,0,1467810369,2009-04-06 22:19:45,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,2009-04-06 22:19:49,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,2009-04-06 22:19:53,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,2009-04-06 22:19:57,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,2009-04-06 22:19:57,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
df_treat.drop(['date', 'flag', 'user'], axis=1, inplace=True)

In [13]:
df_treat

,target,ids,text
0,0,1467810369,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,is upset that he can't update his Facebook by ...
2,0,1467810917,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,my whole body feels itchy and like its on fire
4,0,1467811193,"@nationwideclass no, it's not behaving at all...."
...,...,...,...
1599995,4,2193601966,Just woke up. Having no school is the best fee...
1599996,4,2193601969,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Happy 38th Birthday to my boo of alll time!!! ...


In [14]:
df4 = df_treat[df_treat['target'] == 0]

In [15]:
df4

,target,ids,text
0,0,1467810369,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,is upset that he can't update his Facebook by ...
2,0,1467810917,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,my whole body feels itchy and like its on fire
4,0,1467811193,"@nationwideclass no, it's not behaving at all...."
...,...,...,...
799995,0,2329205009,Sick Spending my day laying in bed listening ...
799996,0,2329205038,Gmail is down?
799997,0,2329205473,rest in peace Farrah! So sad
799998,0,2329205574,@Eric_Urbane Sounds like a rival is flagging y...


In [16]:
print('5 tweets para podermos tratar os dados: \n')

for i in range(0,6):
  print(f'Tweet {i} -> ', df_treat['text'][i], '\n')

5 tweets para podermos tratar os dados: 

Tweet 0 ->  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D 

Tweet 1 ->  is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah! 

Tweet 2 ->  @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds 

Tweet 3 ->  my whole body feels itchy and like its on fire  

Tweet 4 ->  @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.  

Tweet 5 ->  @Kwesidei not the whole crew  



In [17]:
import re


def data_treat(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions and hashtags
    text = re.sub(r'@\w+|#\w+', '', text)

    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [18]:
dft = df[:10]

In [19]:
# Testando função

dft['text'] = dft['text'].apply(data_treat)

C:\Users\Lucas\AppData\Local\Temp\ipykernel_19776\944507329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dft['text'] = dft['text'].apply(data_treat)


In [20]:
dft.head()

,target,ids,date,flag,user,text
0,0,1467810369,2009-04-06 22:19:45,NO_QUERY,_TheSpecialOne_,a thats a bummer you shoulda got david carr of...
1,0,1467810672,2009-04-06 22:19:49,NO_QUERY,scotthamilton,is upset that he cant update his facebook by t...
2,0,1467810917,2009-04-06 22:19:53,NO_QUERY,mattycus,i dived many times for the ball managed to sav...
3,0,1467811184,2009-04-06 22:19:57,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,2009-04-06 22:19:57,NO_QUERY,Karoli,no its not behaving at all im mad why am i her...


In [21]:
df_treat['text'] = df_treat['text'].apply(data_treat)

In [22]:
df_treat

,target,ids,text
0,0,1467810369,a thats a bummer you shoulda got david carr of...
1,0,1467810672,is upset that he cant update his facebook by t...
2,0,1467810917,i dived many times for the ball managed to sav...
3,0,1467811184,my whole body feels itchy and like its on fire
4,0,1467811193,no its not behaving at all im mad why am i her...
...,...,...,...
1599995,4,2193601966,just woke up having no school is the best feel...
1599996,4,2193601969,thewdbcom very cool to hear old walt interviews â
1599997,4,2193601991,are you ready for your mojo makeover ask me fo...
1599998,4,2193602064,happy 38th birthday to my boo of alll time tup...


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Selecionar 150.000 amostras do dataset original
df_subset = df_treat.sample(n=150000, random_state=42)

# Embaralhar e dividir o subset em treino e teste
train_df, test_df = train_test_split(df_subset, test_size=50000, shuffle=True, random_state=42)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Verificar as dimensões
print(f"Tamanho do treino: {train_df.shape}, Tamanho do teste: {test_df.shape}")

# Agora, você pode continuar com a criação dos embeddings e DataLoaders
train_embeddings = train_df['text'].values  # Pegando os embeddings de treino
test_embeddings = test_df['text'].values  # Pegando os embeddings de teste

# Codificando os alvos para valores inteiros
le = LabelEncoder()

train_embeddingsy = le.fit_transform(train_df['target'])  # Codificando as classes de treino
test_embeddingsy = le.transform(test_df['target'])  # Codificando as classes de teste

Tamanho do treino: (100000, 3), Tamanho do teste: (50000, 3)


## Começando treinamento do Modelo

In [25]:
model_path = "ibm-granite/granite-embedding-278m-multilingual"
# Load the Sentence Transformer model
model = SentenceTransformer(model_path)

# encode queries and passages
train_embeddings = model.encode(train_embeddings)

test_embeddings = model.encode(test_embeddings)



In [26]:
from torch.utils.data import TensorDataset, DataLoader


# Certifique-se de que os dados estão no dispositivo certo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Criando Dataset e DataLoader para os dados de teste
X_test = torch.tensor(test_embeddings, dtype=torch.float32).to(device)
y_test = torch.tensor(test_embeddingsy, dtype=torch.long).to(device)
test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Testar iteração dos dados de teste
for X, y in test_dataloader:
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    break  # Apenas para verificar um batch

# Criando Dataset e DataLoader para os dados de treino
X_train = torch.tensor(train_embeddings, dtype=torch.float32).to(device)
y_train = torch.tensor(train_embeddingsy, dtype=torch.long).to(device)
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Testar iteração dos dados de treino
for X, y in train_dataloader:
    print(f"X shape: {X.shape}, y shape: {y.shape}")
    break  # Apenas para verificar um batch



X shape: torch.Size([64, 768]), y shape: torch.Size([64])
X shape: torch.Size([64, 768]), y shape: torch.Size([64])


In [27]:
y_train

tensor([0, 1, 1,  ..., 1, 0, 0], device='cuda:0')

In [28]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definição do modelo SYA3
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()  # Chama o construtor da classe pai (nn.Module)

        # Pilha de camadas lineares com funções de ativação ReLU
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(768, 512),  # Camada totalmente conectada de entrada (768 -> 512 neurônios)
            nn.ReLU(),  # Função de ativação ReLU para introduzir não linearidade
            nn.Linear(512, 512),  # Segunda camada oculta (512 -> 512 neurônios)
            nn.ReLU(),  # Ativação ReLU novamente
            nn.Linear(512, 2),  # Camada de saída (512 -> 10 neurônios, supondo 2 classes)
        )

    def forward(self, x):
        """Define a passagem para frente (forward pass) do modelo"""
        logits = self.linear_relu_stack(x)  # Passa os dados pela pilha de camadas
        return logits  # Retorna os logits (valores brutos antes da ativação softmax)

# Verifica se há uma GPU disponível e define o dispositivo adequado (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instancia o modelo e o move para o dispositivo correto (GPU se disponível, senão CPU)
SentimentAnalysis_Model = NeuralNetwork().to(device)

# Exibe a arquitetura do modelo
print(SentimentAnalysis_Model)


NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=768, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


**O que é a ReLU?**

A ReLU (Rectified Linear Unit) é uma função de ativação amplamente usada em redes neurais. Ela é responsável por introduzir não linearidade no modelo, permitindo que ele aprenda padrões mais complexos.

A fórmula da ReLU é simples:

    ReLU(x) = max(0, x)

Ou seja:

    Se o valor de entrada x for positivo, a saída será x.
    Se o valor de entrada x for negativo, a saída será 0.

**Por que usar a ReLU?**

    Simples e eficiente: A ReLU é muito simples de calcular, o que ajuda a tornar o treinamento mais rápido.
    Evita o problema do gradiente desvanecido: Quando você usa funções como sigmoid ou tanh, elas podem "saturar" (ficar muito próximas de 0 ou 1), fazendo com que o gradiente durante o treinamento diminua muito (desvanecimento do gradiente). A ReLU resolve esse problema, pois para valores positivos, ela retorna uma inclinação constante.
    Efetiva em muitas tarefas: A ReLU tem sido usada com sucesso em uma vasta gama de modelos de deep learning.

Exemplo:

Se tivermos uma entrada x = -2, a ReLU retorna 0. Se a entrada for x = 3, a saída será 3.

**O que é "não linearidade" e por que é importante?**

Não linearidade é a capacidade de um modelo aprender relações complexas entre as entradas e saídas.
Se usássemos apenas operações lineares (por exemplo, multiplicações e somas), o modelo seria equivalente a uma única transformação linear, não importando quantas camadas tivéssemos. Isso limitaria sua capacidade de modelar padrões complexos.

    Ao aplicar funções de ativação não lineares, como a ReLU, o modelo pode combinar essas transformações lineares de forma a capturar relações complexas.
    Exemplo: Imagine que você precisa modelar uma relação curva entre variáveis. Uma simples combinação linear não consegue representar essa curva. Com funções não lineares, o modelo pode aprender essa curvatura e outros padrões complexos presentes nos dados.

In [30]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(SentimentAnalysis_Model.parameters(), lr=1e-3)



**Função de Perda**

A função de perda é uma função matemática que mede o erro do modelo durante o treinamento. O objetivo do treinamento é minimizar esse erro, ou seja, fazer com que o modelo preveja o mais próximo possível da verdade real.

No seu caso, você está usando a função de perda CrossEntropyLoss.

CrossEntropyLoss é usada em problemas de classificação, onde o modelo precisa escolher entre várias classes possíveis (por exemplo, identificar qual é a imagem de um gato, cachorro, etc.).

A função CrossEntropyLoss compara as probabilidades previstas pelo modelo com as probabilidades reais e calcula um valor de erro. Quanto menor o valor da perda, melhor o modelo está se saindo.
Como a CrossEntropyLoss funciona:

    O modelo gera uma probabilidade para cada classe. Se temos 3 classes, o modelo pode prever algo como: [0.2, 0.7, 0.1].
    Se a classe real for a classe 2 (index 1), a CrossEntropyLoss vai penalizar o modelo mais fortemente quanto menor for a probabilidade atribuída à classe correta.

**Otimizador**

O otimizador é o componente que ajusta os pesos do modelo para reduzir a função de perda. Ele basicamente determina como os parâmetros do modelo (os pesos das conexões entre os neurônios) devem ser atualizados para reduzir o erro.

Aqui estamos usando o SGD (Stochastic Gradient Descent) como otimizador.

O que é o SGD?

O SGD é uma versão do gradiente descendente onde a atualização dos pesos é feita com base em apenas um exemplo (ou poucos exemplos) de cada vez, ao invés de usar todos os exemplos no conjunto de dados. Isso torna o processo de otimização mais rápido, embora possa ser um pouco mais ruidoso.
Como o SGD funciona:

    Calcula o Gradiente: O SGD calcula a direção do gradiente da função de perda em relação aos parâmetros do modelo (como os pesos). O gradiente indica em que direção devemos ajustar os parâmetros para reduzir a perda.
    Atualiza os Pesos: O SGD usa a direção do gradiente para atualizar os parâmetros. A atualização é feita com base em uma taxa chamada taxa de aprendizado (learning rate).
        Se a taxa de aprendizado for alta, os pesos serão ajustados em grandes passos.
        Se for baixa, o ajuste será mais sutil.

A fórmula para o gradiente descendente simples é:

    w = w - lr * gradiente, onde:
        w são os pesos do modelo,
        lr é a taxa de aprendizado,
        gradiente é a derivada da função de perda em relação aos pesos.

Parâmetros do Otimizador:

    model.parameters(): Passa todos os parâmetros do modelo (os pesos e vieses dos neurônios) para o otimizador, para que ele possa atualizá-los durante o treinamento.
    lr=1e-3: A taxa de aprendizado, que controla o tamanho dos passos dados durante a atualização dos pesos. 1e-3 é o mesmo que 0.001. Se a taxa de aprendizado for muito alta, o modelo pode "pular" a solução ótima. Se for muito baixa, o treinamento será mais demorado.

Resumo:

    Função de Perda (como CrossEntropyLoss) mede o erro entre as previsões do modelo e as classes reais. O objetivo é minimizar essa perda.
    Otimizador (como SGD) atualiza os pesos do modelo para reduzir a perda, ajustando-os com base no gradiente da função de perda.



In [31]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # Calcula o tamanho do dataset de treino
    model.train() # Treina o modelo
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error -> Preve o resultado e calcula  função de perda
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation -> Otimizador usa a função de perda para ajustar o modelo e ao mesmo tempo minimizar a função de perda
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [32]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # Calcula o tamanho do dataset de teste
    num_batches = len(dataloader) # Calcula a quantidade de batchs (quantidade de dados que serão processados por vez)
    model.eval() # Avaliação do modelo
    test_loss, correct = 0, 0  # Inicializa as variáveis para armazenar a perda total (test_loss) e o número de acertos (correct) com zero
    with torch.no_grad():  # Desativa o cálculo de gradientes para economizar memória e tempo de computação
        for X, y in dataloader:  # Itera sobre o DataLoader, que fornece batches de dados de entrada (X) e rótulos reais (y)
            X, y = X.to(device), y.to(device)  # Move os dados e rótulos para o dispositivo (CPU ou GPU)
            pred = model(X)  # Faz uma previsão usando o modelo com os dados de entrada X
            test_loss += loss_fn(pred, y).item()  # Calcula a perda entre a previsão e o rótulo real, adicionando ao total da perda
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # Conta as previsões corretas (compara a classe com maior probabilidade)
    test_loss /= num_batches  # Calcula a média da perda no conjunto de teste
    correct /= size  # Calcula a acurácia dividindo o número de acertos pelo total de amostras

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [33]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    print(f'\nTraining: \n')
    train(train_dataloader, SentimentAnalysis_Model, loss_fn, optimizer)
    print(f'\nTesting: \n')

    test(test_dataloader, SentimentAnalysis_Model, loss_fn)
print("Done!")

Epoch 1
-------------------------------

Training: 

loss: 0.691761  [   64/100000]
loss: 0.690514  [ 6464/100000]
loss: 0.694211  [12864/100000]
loss: 0.689152  [19264/100000]
loss: 0.693704  [25664/100000]
loss: 0.692391  [32064/100000]
loss: 0.689284  [38464/100000]
loss: 0.693021  [44864/100000]
loss: 0.691416  [51264/100000]
loss: 0.693556  [57664/100000]
loss: 0.693695  [64064/100000]
loss: 0.693706  [70464/100000]
loss: 0.691678  [76864/100000]
loss: 0.692293  [83264/100000]
loss: 0.692623  [89664/100000]
loss: 0.691928  [96064/100000]

Testing: 

Test Error: 
 Accuracy: 50.2%, Avg loss: 0.692970 

Epoch 2
-------------------------------

Training: 

loss: 0.693437  [   64/100000]
loss: 0.693887  [ 6464/100000]
loss: 0.691819  [12864/100000]
loss: 0.695604  [19264/100000]
loss: 0.693179  [25664/100000]
loss: 0.692986  [32064/100000]
loss: 0.693434  [38464/100000]
loss: 0.693924  [44864/100000]
loss: 0.692936  [51264/100000]
loss: 0.693187  [57664/100000]
loss: 0.691977  [64064/1

In [35]:

def programa(texto):
    # Carregar o modelo de embeddings (SentenceTransformer)

    # Cria uma lista com o texto a ser processado
    novos_textos = [texto]

    # Gera os embeddings dos textos (retorna um numpy array)
    novos_embeddings = model.encode(novos_textos)

    # Converte os embeddings para tensor e os move para o dispositivo
    novos_embeddings_tensor = torch.tensor(novos_embeddings, dtype=torch.float32).to(device)

    # Coloca o modelo de classificação em modo de avaliação
    SentimentAnalysis_Model.eval()

    with torch.no_grad():
        # Passa os embeddings pelo modelo de classificação
        outputs = SentimentAnalysis_Model(novos_embeddings_tensor)
        # Se o modelo é de classificação, obtenha a classe prevista
        _, predicted_classes = torch.max(outputs, dim=1)
        predicted_label = le.inverse_transform(predicted_classes.cpu().numpy())

    print(f"Texto: {texto}")
    #print(f"Embeddings gerados: {novos_embeddings_tensor}")
    if predicted_label == 4:
        
       #print(f"Classe prevista: {predicted_label.item()}")
       print(f'Texto positivo!')
    else:
        print(f'Texto negativo')




Texto: I  hate MY LIFE EVERYDAY
Texto negativo


In [36]:
text = input('Escreva sua mensagem: ')
programa(text)

Texto: I love my life
Texto positivo!


In [37]:
torch.save(SentimentAnalysis_Model.state_dict(), "SentimentAnalysis_Model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
